<center>
    <h1> ILI-286 - Laboratorio #3 </h1>
    <h2> Simulando númericamente sistemas dinámicos </h2>
</center>


| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |

## Tabla de contenidos
* [Introducción](#intro)
* [Desarrollo y analisís de resultados:](#desarrollo)
  1. [Simulando Sistemas Caóticos](#se1)
  2. [BVP de orden superior](#se2)
* [Concluciones](#Concluciones)
* [Referencias](#Referencias)
* [Anexos](#Anexos) $\leftarrow$ Comenzar ejecución aquí
  * [Métodos implementados](#an1)
  * [Métodos paralelos](#an2)
  * [Visualización de resultados](#an3)
  * [Implementación de diferencias finitas](#an4)
  * [Utilidades](#an5)

<div id='intro' />
## Introducción
TODO: write something

<div id='desarrollo'/>
## Desarrollo y analisís de resultados

<div id='se1'/>
## 1.- Simulando Sistemas Caóticos
En este laboratorio trabajamos con el Atractor de Lorenz, definido como sigue: 
\begin{align} 
    \frac{dx}{dt} &= \sigma (y-x) \\
    \frac{dy}{dt} &= x(\rho-z) -y \\
    \frac{dz}{dt} &= xy - \beta z 
\end{align}
Además en esta experiencia utilizaremos la configuración $\sigma=10, \rho=28, \beta=\frac{8}{3}$ donde 
las trayectorias se vuelven caóticas. Luego podemos escribir nuestra función en python:

In [ ]:
def lorenz(t, Y):
    return np.array([10*(Y[1] - Y[0]),
                    Y[0]*(28 - Y[2]) - Y[1],
                    Y[0]*Y[1] - (8/3)*Y[2]])

### a) Implementación
Ver [anexo 1](#an1)

### b) Ejecución
Para determinar el maximo $h$ posible debemos tener en cuenta que la región de estabilidad más pequeña
es la del método de euler, además, al estar contenida por las demás, basta con satisfacer solo su condición.

Primero calculamos el Jacobiano de la función de lorenz (caótica) y obtenemos:
$$
\begin{bmatrix}
    -10 & 10 & 0 \\
    28-z_0 & -1 & -x_0 \\
    y_0 & x_0 & -\frac{8}{3}
\end{bmatrix}
$$
Debemos obtener los valores propios de dicha matriz, para determinar el $h$ inicial primero debemos evaluarla
en $Y_0 = [x_0, y_0, z_0]$, luego:

In [ ]:
Y_0 = np.array([1,1,1]) # o cualquiera
A = np.array([
        [-10, 10, 0],
        [28-Y_0[2], -1, -Y_0[0]],
        [Y_0[1], Y_0[0], -8/3]
    ])
ev_A = np.linalg.eigvals(A)
print(ev_A) # que hago con los positivos?!

Para cada uno de los valores propios y un $h$ se debe cumplir que $|1+\lambda h| < 1$, sabemos
$h>0$ por lo que nos queda $h>\frac{-2}{\lambda}$

In [ ]:
max(-2/(ev_A)) # Obtenemos el valor mínimo de h que podemos tomar como el mayor de ésta división.

In [ ]:
t_f = 100
h   = 0.01
t   =  np.linspace(0, t_f, (t_f/h)+1)

dataRK4   = RK4_int(lorenz, Y_0, t, h)
dataMP    = mp_int(lorenz, Y_0, t, h)
dataEuler = euler_int(lorenz, Y_0, t, h)


In [ ]:
# No resultan los animate D: intenté con la transpuesta tmn.
# OJO que cambie el número de frames del código original porque no era int.

#animate_lorenz(dataMP, 1)
animate_lorenz(dataRK4, 1)
#animate_lorenz(dataEuler, 1)

#Un plot rápido para comprobar que está bien
#plt.figure(figsize=(10,6))
#plt.scatter(dataRK4[:,0], dataRK4[:,1])
#plt.show()

### c) Profiling

In [ ]:
t_f = 1000
h   = 0.01
t   = np.linspace(0, t_f, (t_f/h)+1)
#timeit
for name in methods:
    print(name + ":")
    %timeit methods[name](lorenz, Y_0, t, h)

In [ ]:
for name in methods:
    print(name + ":")
    %memit methods[name](lorenz, Y_0, t, h)

### d) Análisis de estabilidad

In [ ]:
hs = [0.1, 0.01, 0.005, 0.001]

### e) Implementación paralela
Ver [anexo 2](#an2)

### f) Simulación de dos trayetorias

### g) Simulación de veinte trayetorias con RK4

<div id='se2'/>
## 2.- BVP de orden superior

### a) Transformación a IVP acoplado
$$ y^{(4)}(x) = -y(x) $$
$$ BC: \{ y(0)= 0, y(2) =1, y'(2)=-1, y''(2)=1 \} $$

Se hace un cambio de variable:
$$
\begin{matrix}
y_1 & = & y \\
y_2 & = & y' \\
y_3 & = & y''\\
y_4 & = & y^{(3)}
\end{matrix} \quad \Longrightarrow \quad
\begin{matrix}
y_1' & = & y' \\
y_2' & = & y'' \\
y_3' & = & y^{(3)}\\
y_4' & = & y^{(4)}
\end{matrix} \quad \Longrightarrow \quad
\begin{matrix}
y_1' & = & y_2 \\
y_2' & = & y_3 \\
y_3' & = & y_4 \\
y_4' & = & -y_1
\end{matrix}
$$

Luego: $ y_1(0)=0, y_2(0) = \alpha , y_3(0) = \beta, y_4(0) = \gamma $

Donde $\alpha$, $\beta$ y $\gamma$ son valores tales que permiten que $y_1(2) = 1$, $y_2(2)=-1$ y $y_3(2)=1$ con algún algoritmo.

### b) Propuesta de cuarta derivada y discretización
Para discretizar la ecuación anterior se debe recurrir a Backward Difference (BD), Forward Difference (FD) y Central Difference (CD). Para aproximar la cuarta derivada se van a necesitar 5 puntos como mínimo para la discretización. 

Lo primero es aproximar las primeras derivadas en los 5 puntos con BD y FD. Luego se aproximarán las segundas derivadas en los puntos intermedios de los 5 puntos (los 3 puntos intermedios). 

Para calcular las segundas derivadas se hará un CD entre la primera derivada del punto anterior calculada con BD y la primera derivada del punto siguiente calculada con FD, así logramos tener las segundas derivadas para los 3 puntos centrales. Tomando como punto central $x$ y intervalos $h$ las derivadas quedan como sigue:

$$ f''(x-h) = \displaystyle \frac{f(x-2h) - 2f(x-h) + f(x)}{h^2} $$
$$ f''(x) = \displaystyle \frac{f(x-h) - 2f(x) + f(x+h)}{h^2} $$
$$ f''(x+h) = \displaystyle \frac{f(x) - 2f(x+h) + f(x+2h)}{h^2} $$

Una vez aproximadas las segundas derivadas, aproximaremos las terceras derivadas. Para ello se utilizará un FD y BW.

$$ f^{(3)}(x-h) = \displaystyle \frac{f''(x) - f''(x-h)}{h} = \displaystyle \frac{f(x+h)-3f(x)+3f(x-h)-f(x-2h)}{h^3} $$
$$ f^{(3)}(x+h) = \displaystyle \frac{f''(x+h) - f''(x)}{h} = \displaystyle \frac{f(x+2h)-3f(x+h)+3f(x)-f(x-h)}{h^3} $$

Finalmente, aproximamos la cuarta derivada en $x$ con CD sobre estas dos derivadas.

$$ f^{(4)}(x) = \displaystyle \frac{f^{(3)}(x+h) - f^{(3)}(x-h)}{2h} = \displaystyle \frac{f(x+2h)-4f(x+h)+6f(x)-4f(x-h)+f(x-2h)}{2h^4} $$

Además, debemos aproximar la segunda derivada en el punto final. Para ello tendremos problemas si usamos CD ya que no tendremos los puntos que vienen más a la derecha, para ellos tendremos que usar 2 veces BD para sacar 2 primeras derivadas y aplicar BD sobre ellas, considerando $N$ intervalos obtendremos:

$$ f''(x_{N}) = \displaystyle \frac{y_N-2y_{N-1}+y_{N-2}}{h^2} $$

Ahora, discretizando nuestro problema con la aproximación conseguida y tomando $N+1$ puntos $(x_0,y_0),(x_1,y_1),\cdots ,(x_{N-1},y_{N-1}),(x_{N},y_{N})$ con $h$ como intervalo obtenemos las siguientes ecuaciones:

\begin{align}
y_0   &= 0 \\
y_{i} &= \displaystyle \frac{6y_{i}-4y_{i+1}+y_{i+2}-4y_{i-1}+y_{i-2}}{2h^4} \\
y_{N} &= 1
\end{align}
Además:
$$ \displaystyle \frac{y_{N}-y_{N-1}}{h} = -1 $$
$$ \displaystyle \frac{y_{N}-2y_{N-1}+y_{N-2}}{h^2} = 1 $$

Reemplazando lo conocido y utilizando $ N + 1$ puntos y $N$ intervalos:

$$ y_i = \displaystyle \frac{6y_i-4y_{i+1}+1-4y_{i-1}}{2h^4} $$
$$ \displaystyle \frac{1-y_{N-1}}{h} = -1 $$
$$ \displaystyle \frac{1-2y_{N-1}+y_{N-2}}{h^2} = 1 $$

Finalmente tenemos 3 ecuaciones y 3 incógnitas resolvibles con algún algoritmo de resolución de sistemas de ecuaciones lineales.

### c) Implementación de diferencias finitas
Ver [Anexo 4](#an4)

Lo primero que se hace para calcular la discretización por medio de diferencias finitas es obtener los valores de $y$ que son fácilmente calculables $y_0, y_N, y_{N-1}, y_{N-2}$ estos valores se obtienen con 4 de las  5 ecuaciones indicadas. En las demás ecuaciones se reemplazan estos valores obtenidos para que dejen de ser incógnitas y pasen a ser valores conocidos y se forma la matriz con las incógnitas restantes. 

Considerando todas las incógnitas y las condiciones $ y(0)=\alpha_1, y'(2)=\alpha_2, y''(2)=\alpha_3, y(2)=\alpha_4 $ la matriz queda así:

$$
\begin{bmatrix} 
    1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots & 0 \\
    -1 & 4 & 2h^4-6 & 4 & -1 & 0 & 0 & 0 &  & 0 \\
    0 & -1 & 4 & 2h^4-6 & 4 & -1 & 0 & 0 &  & 0 \\
    0 & 0 & -1 & 4 & 2h^4-6 & 4 & -1 & 0 &  & 0 \\
    \vdots & & & & & \vdots & & & & \vdots \\
    0 & \cdots & 0 & 0 & 0 &  -1 & 4 & 2h^4-6 & 4 & -1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & 1 & -2 & 1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
\left[ 
\begin{array}{c} 
y_0 \\ 
y_1 \\
y_2 \\
y_3 \\
\vdots \\
y_{N-3} \\
y_{N-2} \\
y_{N-1} \\
y_N \\
\end{array} \right] 
=
\left[ \begin{array}{c}
\alpha_1 \\ 
0 \\
0 \\
0 \\
\vdots \\
0 \\
\alpha_2 \, h \\
\alpha_3 \, h^2 \\
\alpha_4 \\
\end{array} \right]
$$

Luego se calculan los valores de 4 incógnitas como:
\begin{align}
y_0     &= \alpha_1 \\
y_{N-2} &= \alpha_3 \, h^4 + \alpha_4 -2\alpha_2 \, h \\
y_{N-1} &= \alpha_4 - \alpha_2 \, h \\
y_N     &= \alpha_4
\end{align}

La matriz eliminando las 4 incógnitas fácilmente calculables queda así:
$$
\begin{bmatrix} 
    4 & 2h^4-6 & 4 & -1 & 0 & 0 & 0 & 0 & \cdots & 0 \\
    -1 & 4 & 2h^4-6 & 4 & -1 & 0 & 0 & 0 &  & 0 \\
    0 & -1 & 4 & 2h^4-6 & 4 & -1 & 0 & 0 &  & 0 \\
    0 & 0 & -1 & 4 & 2h^4-6 & 4 & -1 & 0 &  & 0 \\
    \vdots & & & & & \vdots & & & & \vdots \\
    0 & \cdots & 0 & 0 & 0 &  -1 & 4 & 2h^4-6 & 4 & -1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & -1 & 4 & 2h^4-6 & 4 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & -1 & 4 & 2h^4-6 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 4
\end{bmatrix}
\left[ 
\begin{array}{c} 
y_1 \\
y_2 \\
y_3 \\
y_4 \\
\vdots \\
y_{N-6} \\
y_{N-5} \\
y_{N-4} \\
y_{N-3} \\
\end{array} \right] 
=
\left[ \begin{array}{c}
\alpha_1 \\ 
0 \\
0 \\
0 \\
\vdots \\
0 \\
y_{N-2}\\
y_{N-1}-4y_{N-2}\\
y_N-4y_{N-1}-(2h^4-6)y_{N-2}\\
\end{array} \right]
$$

Al observar la matriz se puede notar que es casi una matriz triangular superior por lo que es fácilmente convertible en una. Entonces, con el fin de aprovechar las ventajas de una matriz triangular superior convertimos la matriz $A$ en una de ellas por operaciones filas, el vector $b$ obviamente también se ve afectado.

Una vez tenemos nuestro sistema de ecuaciones lineales con una matriz triangular superior lo resolvemos con Backward Substitution obteniendo la solución final.

### d) Ejecución del algoritmo

In [ ]:
#Límite izquierdo de x
x0 = 0
#Límite derecho de x
xN = 2
#Valor de y en el límite izquierdo
y0 = 0
#Valor de y en el límite derecho
yN = 1
#Valor de la derivada en el límite derecho
ydN = -1
#Valor de la segunda derivada en el límite derecho
yddN = 1

for N in [4, 8, 15, 100, 10000]:
    f = fin_dif(N, x0, xN, y0, yN, ydN, yddN)
    plot_2_D(f)

Se puede apreciar claramente como al aumentar la cantidad de intervalos $N$ la presición de la función a aproximar aumenta.

En la primera figura ($N=4$) se puede ver una gráfica muy discretizada donde las líneas no se ven suaves.

En la segunda figura ($N=8$) ya se puede observar mejor la forma de la función, quizás esta discretización ya baste para algunas aplicaciones obteniendo un valor de y bastante cercano al real a pesar de que en la gráfica aún se pueden observar los límites de cada intervalo.

En la tercera figura ($N=15$) con mucho detalle se pueden observar aún los intervalos de discretización pero ya tenemos una función que se puede apreciar correctamente.

En la cuarta ($N=100$) y quinta ($N=10000$) figura ya no se pueden apreciar a simple vista donde comienza y termina un intervalo de discretización, la precisión de las aproximación es bastante alta en estos gráficos.

## Conclusiones

## Referencias
<div id='ref1'\> [1] [Título](www.google.com) Títulos sec. *desc*. Revisado xx/xx/xxxx

## Anexos
Comenzamos cargando los modulos necesarios:

In [ ]:
import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation
#%matplotlib notebook 
%matplotlib nbagg
#%load_ext memory_profiler
#plt.close('all')
#IGNORAR WARNINGS    import warnings    warnings.filterwarnings('ignore')

In [ ]:
#método que permite dibujar gráficos para la pregunda 2.d
def plot_2_D(f):
    #Tamaño de los graficos
    FIGSIZE = (12, 6)
    plt.figure(figsize=FIGSIZE)
    plt.plot(f[:,0], f[:,1])
    plt.title('Gráfico de y(x) ')
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()


<div id='an1'\> 
### Métodos implementados:

In [ ]:
def euler_int(func, y0, t, h):
    ys = np.zeros([t.shape[0],y0.shape[0]])
    ys = np.float64(ys)
    ys[0,:] = y0
    for i in range(t.shape[0]-1):
        ys[i+1,:] = ys[i,:] + h*func(t[i], ys[i,:])
    return ys

In [ ]:
def mp_int(func, y0, t, h):
    h2 = h/2
    ys = np.zeros([t.shape[0],y0.shape[0]])
    ys = np.float64(ys)
    ys[0,:] = y0
    for i in range(t.shape[0]-1):
        k1 = ys[i,:] + h2*func(t[i], ys[i,:])
        ys[i+1,:] = ys[i,:] + h*func(t[i] + h2 , k1)
    return ys

In [ ]:
def RK4_int(func, y0, t, h):
    h2 = h/2
    ys = np.zeros([t.shape[0],y0.shape[0]])
    ys = np.float64(ys)
    ys[0,:] = y0
    for i in range(t.shape[0]-1):
        k1 = func(t[i], ys[i,:])
        k2 = func(t[i] + h2, ys[i,:] + h2*k1)
        k3 = func(t[i] + h2, ys[i,:] + h2*k2)
        k4 = func(t[i] + h,  ys[i,:] + h*k3)
        ys[i+1,:] = ys[i,:] + (h/6)*(k1 + 2*(k2 + k3) + k4)
    return ys

In [ ]:
methods = {"Euler": euler_int, "MidPoint": mp_int, "RK4": RK4_int}

In [ ]:
#test 1-D
y0 = np.array([1])
tf = 4
h  = 1
t  = np.linspace(0, tf, (tf/h)+1)
f  = lambda t,y: np.e**t

#print(euler_int(f, y0, t, h)[-1,:])
#print(mp_int(f, y0, t, h)[-1,:])
#print(RK4_int(f, y0, t, h)[-1,:])
#print(f(4,0))

#test N-D
y02 = np.array([1,2])
tf = 4
h  = 1
t  = np.linspace(0, tf, (tf/h)+1)
f2 = lambda t,y: np.array([t+y[0], y[1]+y[0]])

#print("Euler")
#print(euler_int(f2, y02, t, h)[-1,:])
#print("MP")
#print(mp_int(f2, y02, t, h)[-1,:])
#print("RK4")
#print(RK4_int(f2, y02, t, h)[-1,:])

<div id='an2'\> 
### Métodos paralelos:

In [ ]:
#Retorna una función que ejecuta la función original en una matriz
def mk_parallel_func(func):
    def new_func(t, f, r): #r es un array ya creado, con las dimensiones correctas.
        for i in range(r.shape[1]):
            r[:,i] = func(t, f[:,i])
        return r
    return new_func

In [ ]:
def parallel_euler_int(func, y0, t, h):
    ys = np.float64(np.zeros([t.shape[0], y0.shape[1], y0.shape[0]]))
    rf = np.float64(np.zeros([y0.shape[1], y0.shape[0]]))
    ys[0,:,:] = y0.T
    vf = mk_parallel_func(func)
    for i in range(t.shape[0]-1):
        #for j in range(y0.shape[0]):
        #    rf[:,j] = func(t[i], ys[i,:,j])
        ys[i+1,:,:] = ys[i,:,:] + h * vf(t[0], ys[i,:,:], rf)#rf
    return ys

In [ ]:
def parallel_mp_int(func, y0, t, h):
    h2 = h/2
    ys = np.float64(np.zeros([t.shape[0], y0.shape[1], y0.shape[0]]))
    rf = np.float64(np.zeros([y0.shape[1], y0.shape[0]]))
    ys[0,:,:] = y0.T
    vf = mk_parallel_func(func)
    for i in range(t.shape[0]-1):
        k1 = ys[i,:,:] + h2*vf(t[i], ys[i,:,:], rf)
        ys[i+1,:,:] = ys[i,:,:] + h*vf(t[i] + h2 , k1, rf)
    return ys

In [ ]:
def parallel_RK4_int(func, y0, t, h):
    h2 = h/2
    ys = np.float64(np.zeros([t.shape[0], y0.shape[1], y0.shape[0]]))
    rf = np.float64(np.zeros([y0.shape[1], y0.shape[0]]))
    ys[0,:,:] = y0.T
    vf = mk_parallel_func(func)
    for i in range(t.shape[0]-1):
        k1 = np.copy( vf(t[i], ys[i,:,:], rf) )
        k2 = np.copy( vf(t[i] + h2, ys[i,:,:] + h2*k1, rf) )
        k3 = np.copy( vf(t[i] + h2, ys[i,:,:] + h2*k2, rf) )
        k4 = np.copy( vf(t[i] + h,  ys[i,:,:] + h*k3,  rf) )
        ys[i+1,:,:] = ys[i,:,:] + (h/6)*(k1 + 2*(k2 + k3) + k4)
    return ys

In [ ]:
#Test parallel

lns = np.linspace(0, 1, 5)
c1  = np.array([1,1,1])
c2  = np.array([2,2,2])
c3  = np.array([[1,1,1],[2,2,2]])
#print(RK4_int(lorenz, c1, lns, .25)[1,:])
print(RK4_int(lorenz, c2, lns, .25))
print(parallel_RK4_int(lorenz, c3, lns, .25)[:,:,1])

<div id='an3'\> 
### Visualización de resultados

In [ ]:
"""
-> x_t
   arreglo con las posiciones (x,y,z) de las trayectorias. Este puede ser
   de dos dimensiones (len(t), 3) para una sola trayectoria, o puede ser 
   de tres dimensiones (len(t), 3, N_trajectories) para mas de una trayectoria
-> N_trajectories
   numero de trayectorias a visualizar
-> xlim,ylim,zlim
   limites en los ejes x,y,z de la animacion
-> rotate
   rotar mientras anima para mejores perspectivas
"""
def animate_lorenz(x_t, N_trajectories, xlim=(-20,70), ylim=(-35,35), zlim=(-35,35), rotate=False):
    #setting it to correct format
    if x_t.ndim==2:
        x_t = np.array([x_t])
    elif x_t.ndim==3: 
        x_t = np.rot90(x_t).T
    else:
        return -1
    #setting the number of frames
    frames = max(x_t.shape)
    frames /= 2
    frames = int(frames)
    #set up figure & 3D axis for animation
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1], projection='3d')
    #ax.axis('off')

    #choose a different color for each trajectory
    colors = plt.cm.jet(np.linspace(0, 1, N_trajectories))

    #set up lines and points
    lines = sum([ax.plot([], [], [], '-', c=c)
                 for c in colors], [])
    pts = sum([ax.plot([], [], [], 'o', c=c)
               for c in colors], [])

    #prepare the axes limits
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_zlim(zlim)

    #set point-of-view: specified by (altitude degrees, azimuth degrees)
    ax.view_init(30, 0)

    #initialization function: plot the background of each frame
    def init():
        for line, pt in zip(lines, pts):
            line.set_data([], [])
            line.set_3d_properties([])

            pt.set_data([], [])
            pt.set_3d_properties([])
        return lines + pts

    #animation function.  This will be called sequentially with the frame number
    def animate(i):
        # we'll step two time-steps per frame.  This leads to nice results.
        i = (2 * i) % x_t.shape[1]

        for line, pt, xi in zip(lines, pts, x_t):
            x, y, z = xi[:i].T
            line.set_data(x, y)
            line.set_3d_properties(z)

            pt.set_data(x[-1:], y[-1:])
            pt.set_3d_properties(z[-1:])

        if rotate: ax.view_init(30, 0.3 * i)
        fig.canvas.draw()
        return lines + pts

    #instantiate the animator.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=frames, interval=30, blit=True, repeat=False)
    #show it
    ax.set_title('Lorenz Simulation')
    ax.set_xlabel('x(t)')
    ax.set_ylabel('y(t)')
    ax.set_zlabel('z(t)')
    plt.show()

<div id='an4'\> 
### Implementación de diferencias finitas

In [ ]:
def fin_dif(N, x0, xN, y0, yN, ydN, yddN):
    if (N<4):
        print('Error: Se necesita al menos 4 intervalos (5 puntos).');
        return;
    #inicializando vector de y con ceros.
    y = np.zeros([N+1,1])
    #tamaño del intervalo
    h = (xN-x0)/N;
    x = np.linspace(x0, xN, N+1);
    
    #estableciendo valores conocidos
    y[0] = y0;
    y[N] = yN;
    y[N-1] = y[N]-ydN*h
    y[N-2] = yddN*h**4+yN-2*ydN*h;
    
    M = (N+1)-4;
    A = np.zeros([M, M]);
    b = np.zeros([M, 1]);
    
    #creando matriz A
    for i in range(M):
        if (i >= 1):
            A[i,i-1] = -1;
        A[i,i] = 4;
        if (i <= M-2):
            A[i,i+1] = 2*h**4-6;
        if (i <= M-3):
            A[i,i+2] = 4;
        if (i <= M-4):
            A[i,i+3] = -1;
            
    #creando vector b
    b[0] = y[0];
    if (M >= 3):
        b[M-3] = b[M-3] + y[N-2];
    if (M >= 2):
        b[M-2] = b[M-2] + y[N-1] - 4*y[N-2];
    b[M-1] = b[M-1] + y[N] - 4*y[N-1] - (2*h**4-6)*y[N-2];
    
    #convirtiendo la matriz A en triangular superior
    for i in range(M-1):
        A[i+1, :] = A[i+1, :]+A[i, :]/A[i,i];
        b[i+1] = b[i+1]+b[i]/A[i,i];
    
    # Resolviendo el sistema de ecuaciones lineales con backward substitution
    for i in range(M-1,-1,-1):
        j = i+1;
        if (i== M-1):
            y[j] = b[i]/A[i,i]
        elif (i== M-2):
            y[j] = (b[i]-A[i,i+1]*y[j+1])/A[i,i];
        elif (i== M-3):
            y[j] = (b[i]-A[i,i+1]*y[j+1]-A[i,i+2]*y[j+2])/A[i,i];
        else:
            y[j] = (b[i]-A[i,i+1]*y[j+1]-A[i,i+2]*y[j+2]-A[i,i+3]*y[j+3])/A[i,i];
            
    #formateando output
    f = np.zeros([N+1, 2]);
    f[:,0] = x;
    f[:,1] = y[:,0];
    return f;

<div id='an5'\> 
### Utilidades